<a href="https://colab.research.google.com/github/gustavolio/AI-studies/blob/main/text_mining/Minera%C3%A7%C3%A3o_de_Textos_Parte_1_Exemplo_3_(informa%C3%A7%C3%A3o_geogr%C3%A1fica).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando Bibliotecas


In [ ]:
!pip install geocoder
import geocoder
!pip install haversine
from haversine import haversine, Unit
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor
from polyglot.downloader import downloader
from polyglot.text import Text
!polyglot download ner2.pt
!polyglot download ner2.en
import pandas as pd
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import numpy as np
import networkx as nx
!pip install plotly.express
from plotly import graph_objs as go
import folium


     |████████████████████████████████| 102kB 2.3MB/s 
     |████████████████████████████████| 133kB 2.6MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52559 sha256=a2716b573c863163e9e306d873ec92473ba85a6107a69a765922c389e8d4a634
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b83a305f39ab047986d9da
Successfully built polyglot
     |████████████████████████████████| 235kB 2.7MB/s 
  Created wheel for pyicu: filename=PyICU-2.5-cp36-cp36m-linux_x86_64.whl size=1252664 sha256=1c331878f8a03427cc5f02c5dfca4adaa463d2bca47469eb3f9e13cb85814b10
  Stored in directory: /root/.cache/pip/wheels/07/3a/28/09f90c38785945ddf9af61b7add1aa62a740f40e259626ef3a
Successfully built pyicu
     |████████████████████████████████| 41.4MB 99kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp36-cp36m-linux_x86_64.whl size=9833551 sha256=c4194f7020a11dd48203f7bff4747743ec3549a545f1ca890f790eb367008da5
  Stored in directory: /root/.cac

In [ ]:
!polyglot download embeddings2.pt

[polyglot_data] Downloading package embeddings2.pt to
[polyglot_data]     /root/polyglot_data...


# Extraindo Entidades Nomeadas

In [ ]:
# identificando entidades nomeadas em textos
text = Text("O curso de mineração de dados não estruturados na USP de São Carlos - SP está ocorrendo hoje")
text.entities



[I-ORG(['USP']), I-LOC(['São', 'Carlos']), I-LOC(['SP'])]

# Filtrando Entidades Nomeadas sobre Informação Geográfica

In [ ]:
# extraindo entidades geográficas (tipo I-LOC)
text = Text("A escola avançada de big data na USP de São Carlos - SP está ocorrendo hoje")
loc = ""
for ent in text.entities:
  if ent.tag=='I-LOC':
    for s in ent:
      loc += s+" "
print(loc)

São Carlos SP 


# Aplicando Geocoder (identificando lat/lng)

In [ ]:
# extraindo latitude / longitude
print("Geocoding loc: "+loc)
g = geocoder.geonames(loc, maxRows=1, key='3eabda_1')
for result in g:
  print("Country: "+result.country)
  print("State: "+result.state)
  print("Address: "+result.address)
  print("Lat: "+result.lat)
  print("Lng: "+result.lng)

Geocoding loc: São Carlos SP 
Country: Brazil
State: São Paulo
Address: São Carlos
Lat: -22.0175
Lng: -47.89083


# Calculando distância entre localizações (usando Haversine)

In [ ]:
# calculando distancia haversine entre duas localidades
loc1 = "São Carlos, SP"
loc2 = "Jales, SP"
g1 = geocoder.geonames(loc1, maxRows=1, key='3eabda_1')
g2 = geocoder.geonames(loc2, maxRows=1, key='3eabda_1')
dist_km = haversine( (float(g1[0].lat),float(g1[0].lng)), (float(g2[0].lat),float(g2[0].lng)))
print("Dist (km) = "+str(dist_km))

Dist (km) = 337.0645431239471


In [ ]:
import urllib.parse
# obtendo dataset com uma amostra de textos

query = "febre amarela" # query para consultar
query = urllib.parse.quote(query)

pd.set_option('display.max_colwidth', -1)
dataset = pd.read_csv('http://websensors.net.br/minicurso/2019/eventos-br-2017.php?q='+query,sep='\t', lineterminator='\n')
dataset

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



,event_id,date,title,local,url,lat,lng
0,b4b29e9b5537c1896129029bd976438c,2017-01-05 15:27:28,"Após 2ª morte por febre amarela em SP, vacinação é intensificada - Globo.com",Ribeirao Preto (SP),http://g1.globo.com/sp/ribeirao-preto-franca/noticia/2017/01/apos-2-morte-por-febre-amarela-em-sp-vacinacao-e-intensificada.html,-21.17,-47.81
1,125bf6fbeef15b17a1d4837127129bbe,2017-01-08 16:04:51,Casos de febre amarela em macacos na região preocupam ... - Globo.com,Ribeirao Preto (SP),http://g1.globo.com/sao-paulo/sao-jose-do-rio-preto-aracatuba/alerta-dengue/noticia/2017/01/avanco-da-febre-amarela-na-regiao-preocupa-moradores-e-autoridades.html,-21.17,-47.81
2,aff14d48f09878b31e0575971fc8fec2,2017-01-08 14:04:51,G1 - Casos de febre amarela em macacos na região preocupam ... - Globo.com,Sao Jose do Rio Preto (SP),http://g1.globo.com/sao-paulo/sao-jose-do-rio-preto-aracatuba/alerta-dengue/noticia/2017/01/avanco-da-febre-amarela-na-regiao-preocupa-moradores-e-autoridades.html,-20.82,-49.37
3,ca7ca58720fc9a2338894095ba6409f0,2017-01-05 13:27:28,"Após 2ª morte por febre amarela em SP, vacinação é intensificada - Globo.com",Sao Jose do Rio Preto (SP),http://g1.globo.com/sp/ribeirao-preto-franca/noticia/2017/01/apos-2-morte-por-febre-amarela-em-sp-vacinacao-e-intensificada.html,-20.82,-49.37
4,2a6e78e1226fc9c5f1c9061febf175d1,2017-01-09 14:57:25,Mortes por febre amarela põem em alerta cidades do norte paulista - Istoe,Sao Jose do Rio Preto (SP),http://istoe.com.br/mortes-por-febre-amarela-poem-em-alerta-cidades-do-norte-paulista/,-20.82,-49.37
...,...,...,...,...,...,...,...
295,eb1bf464427f72b429d4d026cd03c15a,2017-01-16 13:26:34,Casos de febre amarela dobram na cidade mais afetada por surto ... - Jornal O Globo,Teofilo Otoni (MG),http://oglobo.globo.com/sociedade/saude/casos-de-febre-amarela-dobram-na-cidade-mais-afetada-por-surto-em-mg-20780480,-17.85,-41.50
296,4011697d8b30ee8b4ef7eba723b4d123,2017-01-16 18:34:04,"Mortes suspeitas por febre amarela sobem para 47 em Minas, diz governo - Globo.com",Guarapari (ES),http://news.google.com/news/url?sa=t&amp;fd=R&amp;ct2=pt-BR_br&amp;usg=AFQjCNGndFS8CsOVRLxukpOQSm61GvywHg&amp;clid=c3a7d30bb8a4878e06b80cf16b898331&amp;cid=52779918976016&amp;ei=UlN9WODZK8mhzALDvQU&amp;url=http://g1.globo.com/minas-gerais/noticia/mortes-suspeitas-por-febre-amarela-sobem-para-47-em-minas-diz-governo.ghtml,-20.65,-40.51
297,cb4d96cd6373d865ca0e3d4c71689fff,2017-01-16 21:25:40,Áreas endêmicas de Febre Amarela são monitoradas no Maranhão,Sao Luis (MA),http://g1.globo.com/ma/maranhao/noticia/2017/01/areas-endemicas-de-febre-amarela-sao-monitoradas-no-maranhao.html,-2.53,-44.30
298,9e5608325882b701c343427999d2c4e9,2017-01-16 18:30:41,"Casos suspeitos de febre amarela em MG sobem para 152, com 47 mortes - Jornal O Globo",Coronel Fabriciano (MG),http://oglobo.globo.com/sociedade/saude/casos-suspeitos-de-febre-amarela-em-mg-sobem-para-152-com-47-mortes-20783045,-19.51,-42.62


In [ ]:
# filtrando dados de interesse
dataset = dataset[['title','local','lat','lng']]
dataset

,title,local,lat,lng
0,"Após 2ª morte por febre amarela em SP, vacinação é intensificada - Globo.com",Ribeirao Preto (SP),-21.17,-47.81
1,Casos de febre amarela em macacos na região preocupam ... - Globo.com,Ribeirao Preto (SP),-21.17,-47.81
2,G1 - Casos de febre amarela em macacos na região preocupam ... - Globo.com,Sao Jose do Rio Preto (SP),-20.82,-49.37
3,"Após 2ª morte por febre amarela em SP, vacinação é intensificada - Globo.com",Sao Jose do Rio Preto (SP),-20.82,-49.37
4,Mortes por febre amarela põem em alerta cidades do norte paulista - Istoe,Sao Jose do Rio Preto (SP),-20.82,-49.37
...,...,...,...,...
295,Casos de febre amarela dobram na cidade mais afetada por surto ... - Jornal O Globo,Teofilo Otoni (MG),-17.85,-41.50
296,"Mortes suspeitas por febre amarela sobem para 47 em Minas, diz governo - Globo.com",Guarapari (ES),-20.65,-40.51
297,Áreas endêmicas de Febre Amarela são monitoradas no Maranhão,Sao Luis (MA),-2.53,-44.30
298,"Casos suspeitos de febre amarela em MG sobem para 152, com 47 mortes - Jornal O Globo",Coronel Fabriciano (MG),-19.51,-42.62


# Plotando a informação geográfica dos textos

In [ ]:
# plotando eventos no mapa
m = folium.Map([-14.5931291,-56.6985808], zoom_start=4)

for node in G.nodes():
  lat = dataset.iloc[node,:]['lat']
  lng = dataset.iloc[node,:]['lng']
  text = dataset.iloc[node,:]['title']
  folium.Marker([lat, lng], popup=text).add_to(m)

m

In [ ]:
# plotando eventos no mapa com heatmap
from folium.plugins import HeatMap

m = folium.Map([-14.5931291,-56.6985808], zoom_start=4)
data_heatmap = []
for node in G.nodes():
  lat = dataset.iloc[node,:]['lat']
  lng = dataset.iloc[node,:]['lng']
  data_heatmap.append([lat,lng,1])

HeatMap(data_heatmap).add_to(m)

m